## Importing packages

In [1]:
import requests # request accesing pages
from bs4 import BeautifulSoup # to scrape html code from pages
import re # to cleanup some data points - searches count-
from __future__ import division # needed for division
import pandas as pd # to create a data frame
import numpy as np # generate random times for pauses between requests 
import time # package for pausing code
from time import gmtime, strftime

# Defining a function to transform the desired keywords from a dataframe into a dictionary
def dataframeToDict(df):
    ncol = df.columns
    skilldict = {}
    for c in ncol:
        cvalues = df[c].values
        skilldict[c] = cvalues[~pd.isnull(cvalues)]
    return skilldict

## Inputs & loading csvs

In [2]:

#number of results wanted per page, min = 10, max = 50, Bigger value > faster code
page_count = 50 

#importing searches & keywords

searches = pd.read_csv('searches.csv')
keywords = pd.read_csv('keywords.csv')
keywords = dataframeToDict(keywords)
seperator = "_" 

start_time = strftime("%Y-%m-%d %H:%M:%S", gmtime())
start = time.time()

# Phase 1: Scraping job links

In [3]:
p1_start = time.time()

# Creating the Data Frame that will hold the scraped data
df_final = pd.DataFrame(columns=["Search", "Title","Location","Company","Salary", "Summary", "Post_Date", "Link"])
print "We are going to scrape jobs for " + str(searches.shape[0]) + " searches:"
print " "
failed = 0
for search in range(0,searches.shape[0], 1):
    df = pd.DataFrame(columns=["Search", "Title","Location","Company","Salary", "Summary", "Post_Date", "Link"])
    url = "https://www.indeed.com/jobs?q=" + searches.job_title[search].lower().replace(' ', '+') + \
    "&l=" + searches.job_location[search].lower().replace(' ', '+') + "&" + 'radius=' + str(searches.search_radius[search]) + "&jt=" + searches.job_type[search] + \
    "&filter=0&limit=" + str(page_count)
    # making sure job title does not have " "
    searches.job_title[search] = searches.job_title[search].replace('"', '')
    
    # request and scrape first page results
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")
    
    # scraping the total number of jobs for this search
    try:
        results_count = soup.select('#searchCount')
        # cleaning up the scraped string
        results_count = re.sub('\D', '', str(results_count))[1::]
        results_count = int(results_count)
    except:
        results_count = 'NO'
        failed = failed + 1

    
    print "-------------------------"
    print ""
    if results_count > 1000 and results_count <> "NO":
        print "There are " + str(results_count) +  " jobs for your [  " + str(search + 1) + " / " +  str(str(searches.shape[0])) + "  ] search!, but we can only scrape 1000."
    else:
        print "There are " + str(results_count) +  " jobs for your [  " + str(search + 1) + " / " +  str(str(searches.shape[0])) + "  ] search!"
    
    print ""
    print "link of this search: " + url
    print ""
    

    # to make sure we only scrape the exact number of job posts, otherwise we will have many duplicates

    if results_count < 1000:
        target = results_count
    else:
        target = 1000

    # Loop to request different results pages
    if results_count <> 'NO':

        for page in range(0, target, page_count):
            if page is not 0:
                progress =  int(page/target * 100)
                print "Scraped " + str(page) + " jobs [ " + str(progress) + "% ]  [  " + str(search + 1) + " / " +  str(str(searches.shape[0])) + "  ]"

            else:

                print "Initializing Scraping " + str(target) + " " + str(searches.job_title[search]) +" jobs in " + searches.job_location[search] + "!" + \
                "  [  " + str(search + 1) + " / " +  str(str(searches.shape[0])) + "  ]"
                print " "
            new_url = url + "&start=" + str(page)
            html = requests.get(new_url)
            soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")

        # Loop to scrape job title, location, company, salary, synopsis, post date, link

            for each in soup.find_all(class_= "result"):
                sponsor = each.find(class_='sponsoredGray') 

                if sponsor is None: #ignore any post that is not organic (sponsored)

                    try:
                        date = each.find('span', {'class':'date' }).text.replace('\n', '')
                    except:
                        date = 'None'
                    try:
                        joblink = each.find(class_= 'turnstileLink').get('href')
                        link = 'https://www.indeed.com' + joblink
                    except:
                        link = 'None'
                    try: 
                        title = each.find(class_='jobtitle').text.replace('\n', '')
                    except:
                        title = 'None'
                    try:
                        location = each.find('span', {'class':'location' }).text.replace('\n', '')
                    except:
                        location = 'None'
                    try: 
                        company = each.find(class_='company').text.replace('\n', '')
                    except:
                        company = 'None'
                    try:
                        salary = each.find('span', {'class':'no-wrap'}).text.replace('\n', '')
                    except:
                        salary = 'None'
                    try:
                        summary = each.find('span', {'class':'summary'}).text.replace('\n', '')
                    except:
                        summary = 'None'
        # Append the scraped datapoints into a record in the Data Frame           

                    df = df.append({'Title':title, 'Location':location, 'Company':company,
                                    'Salary':salary, 'Summary':summary, 'Post_Date':date, 'Link':link,
                                    'Search':searches.job_title[search]}, ignore_index=True)

        # Pausing the loop for a random value in a normal distribution with a mean of 1 seconds, imitate human browsing     

            pausetime = int(np.abs(np.random.randn(1) + 1))  
            if page <> 0:
                print "Pausing for " + str(pausetime) + " seconds"
            print "-------------------------"
            time.sleep(pausetime)
    else:
        print "Skipping  [  " + str(search + 1) + " / " +  str(str(searches.shape[0])) + "  ] search!"
    df_final = pd.concat([df_final,df])
    progress =  int(df.shape[0]/target * 100)  

    if results_count <> "NO":
        print " "
        print "Finished Scraping " + str(df.shape[0]) + " jobs [ " + str(progress) + "% ]  [  " + str(search + 1) + " / " +  str(searches.shape[0]) + "  ]  [  Total: " + str(df_final.shape[0])  + " jobs  ]" 
        print " "
    
        if df.shape[0] <> target:
            print "Dont worry if the % or number of jobs dont match, Indeed changes the search results on the go as you browse!"
    
    print " "
print " "
print " "
print "-------------------------"
print "Done with the first phase"
scraped = df_final.shape[0]
if failed <> 0:
    print "There were " + str(failed) + " searches that had no results, so we only scraped jobs for " + str(searches.shape[0] - failed) + " out of the " +  str(searches.shape[0]) + " searches." 
print "Total jobs scraped: " + str(scraped) + " jobs."

df_final.drop_duplicates(keep='first', subset="Link", inplace=True)

duplicates  = scraped - df_final.shape[0]

print "There were " + str(duplicates) + " duplicated jobs that we removed." 
print "So we only saved " + str(df_final.shape[0]) + " jobs."
df_final.reset_index(drop=True, inplace=True)
df1 = df_final

p1_duration = time.strftime("%H:%M:%S", time.gmtime(time.time() - p1_start))
print "Duration: " + p1_duration

We are going to scrape jobs for 1240 searches:
 


C:\Users\Zeus\Anaconda2\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


-------------------------

There are 32 jobs for your [  1 / 1240  ] search!

link of this search: https://www.indeed.com/jobs?q=actuarial+analyst&l=san+francisco&radius=5&jt=all&filter=0&limit=50

Initializing Scraping 32 Actuarial Analyst jobs in San Francisco!  [  1 / 1240  ]
 
-------------------------
 
Finished Scraping 31 jobs [ 96% ]  [  1 / 1240  ]  [  Total: 31 jobs  ]
 
Dont worry if the % or number of jobs dont match, Indeed changes the search results on the go as you browse!
 
-------------------------

There are 1 jobs for your [  2 / 1240  ] search!

link of this search: https://www.indeed.com/jobs?q=actuarial+analyst&l=san+jose&radius=5&jt=all&filter=0&limit=50

Initializing Scraping 1 Actuarial Analyst jobs in San Jose!  [  2 / 1240  ]
 
-------------------------
 
Finished Scraping 1 jobs [ 100% ]  [  2 / 1240  ]  [  Total: 32 jobs  ]
 
 
-------------------------

There are 3 jobs for your [  3 / 1240  ] search!

link of this search: https://www.indeed.com/jobs?q=act

-------------------------

There are 39 jobs for your [  20 / 1240  ] search!

link of this search: https://www.indeed.com/jobs?q=actuarial+analyst&l=south+san+francisco&radius=5&jt=all&filter=0&limit=50

Initializing Scraping 39 Actuarial Analyst jobs in South San Francisco!  [  20 / 1240  ]
 
-------------------------
 
Finished Scraping 39 jobs [ 100% ]  [  20 / 1240  ]  [  Total: 320 jobs  ]
 
 
-------------------------

There are 103 jobs for your [  21 / 1240  ] search!

link of this search: https://www.indeed.com/jobs?q=allocation+analyst&l=san+francisco&radius=5&jt=all&filter=0&limit=50

Initializing Scraping 103 Allocation Analyst jobs in San Francisco!  [  21 / 1240  ]
 
-------------------------
Scraped 50 jobs [ 48% ]  [  21 / 1240  ]
Pausing for 1 seconds
-------------------------
Scraped 100 jobs [ 97% ]  [  21 / 1240  ]
Pausing for 2 seconds
-------------------------
 
Finished Scraping 103 jobs [ 100% ]  [  21 / 1240  ]  [  Total: 423 jobs  ]
 
 
----------------------

-------------------------

There are 1 jobs for your [  39 / 1240  ] search!

link of this search: https://www.indeed.com/jobs?q=allocation+analyst&l=hayward&radius=5&jt=all&filter=0&limit=50

Initializing Scraping 1 Allocation Analyst jobs in Hayward!  [  39 / 1240  ]
 
-------------------------
 
Finished Scraping 1 jobs [ 100% ]  [  39 / 1240  ]  [  Total: 871 jobs  ]
 
 
-------------------------

There are 4 jobs for your [  40 / 1240  ] search!

link of this search: https://www.indeed.com/jobs?q=allocation+analyst&l=south+san+francisco&radius=5&jt=all&filter=0&limit=50

Initializing Scraping 4 Allocation Analyst jobs in South San Francisco!  [  40 / 1240  ]
 
-------------------------
 
Finished Scraping 4 jobs [ 100% ]  [  40 / 1240  ]  [  Total: 875 jobs  ]
 
 
-------------------------

There are 2288 jobs for your [  41 / 1240  ] search!, but we can only scrape 1000.

link of this search: https://www.indeed.com/jobs?q=analyst&l=san+francisco&radius=5&jt=all&filter=0&limit=50


...

-------------------------

There are 105 jobs for your [  1178 / 1240  ] search!

link of this search: https://www.indeed.com/jobs?q=financial+analyst&l=pleasanton&radius=5&jt=all&filter=0&limit=50

Initializing Scraping 105 Financial Analyst jobs in Pleasanton!  [  1178 / 1240  ]
 
-------------------------
Scraped 50 jobs [ 47% ]  [  1178 / 1240  ]
Pausing for 0 seconds
-------------------------
Scraped 100 jobs [ 95% ]  [  1178 / 1240  ]
Pausing for 1 seconds
-------------------------
 
Finished Scraping 105 jobs [ 100% ]  [  1178 / 1240  ]  [  Total: 109397 jobs  ]
 
 
-------------------------

There are 15 jobs for your [  1179 / 1240  ] search!

link of this search: https://www.indeed.com/jobs?q=financial+analyst&l=hayward&radius=5&jt=all&filter=0&limit=50

Initializing Scraping 15 Financial Analyst jobs in Hayward!  [  1179 / 1240  ]
 
-------------------------
 
Finished Scraping 15 jobs [ 100% ]  [  1179 / 1240  ]  [  Total: 109412 jobs  ]
 
 
-------------------------

There

Scraped 50 jobs [ 30% ]  [  1189 / 1240  ]
Pausing for 2 seconds
-------------------------
Scraped 100 jobs [ 60% ]  [  1189 / 1240  ]
Pausing for 0 seconds
-------------------------
Scraped 150 jobs [ 90% ]  [  1189 / 1240  ]
Pausing for 2 seconds
-------------------------
 
Finished Scraping 165 jobs [ 100% ]  [  1189 / 1240  ]  [  Total: 111487 jobs  ]
 
 
-------------------------

There are 111 jobs for your [  1190 / 1240  ] search!

link of this search: https://www.indeed.com/jobs?q=marketing+analyst&l=san+mateo&radius=5&jt=all&filter=0&limit=50

Initializing Scraping 111 Marketing Analyst jobs in San Mateo!  [  1190 / 1240  ]
 
-------------------------
Scraped 50 jobs [ 45% ]  [  1190 / 1240  ]
Pausing for 1 seconds
-------------------------
Scraped 100 jobs [ 90% ]  [  1190 / 1240  ]
Pausing for 0 seconds
-------------------------
 
Finished Scraping 111 jobs [ 100% ]  [  1190 / 1240  ]  [  Total: 111598 jobs  ]
 
 
-------------------------

There are 28 jobs for your [  119

-------------------------
 
Finished Scraping 40 jobs [ 100% ]  [  1206 / 1240  ]  [  Total: 112532 jobs  ]
 
 
-------------------------

There are 18 jobs for your [  1207 / 1240  ] search!

link of this search: https://www.indeed.com/jobs?q=hr+analyst&l=alameda&radius=5&jt=all&filter=0&limit=50

Initializing Scraping 18 HR analyst jobs in Alameda!  [  1207 / 1240  ]
 
-------------------------
 
Finished Scraping 19 jobs [ 105% ]  [  1207 / 1240  ]  [  Total: 112551 jobs  ]
 
Dont worry if the % or number of jobs dont match, Indeed changes the search results on the go as you browse!
 
-------------------------

There are 2 jobs for your [  1208 / 1240  ] search!

link of this search: https://www.indeed.com/jobs?q=hr+analyst&l=san+rafael&radius=5&jt=all&filter=0&limit=50

Initializing Scraping 2 HR analyst jobs in San Rafael!  [  1208 / 1240  ]
 
-------------------------
 
Finished Scraping 2 jobs [ 100% ]  [  1208 / 1240  ]  [  Total: 112553 jobs  ]
 
 
-------------------------

T

Pausing for 0 seconds
-------------------------
Scraped 100 jobs [ 28% ]  [  1222 / 1240  ]
Pausing for 1 seconds
-------------------------
Scraped 150 jobs [ 42% ]  [  1222 / 1240  ]
Pausing for 1 seconds
-------------------------
Scraped 200 jobs [ 57% ]  [  1222 / 1240  ]
Pausing for 0 seconds
-------------------------
Scraped 250 jobs [ 71% ]  [  1222 / 1240  ]
Pausing for 0 seconds
-------------------------
Scraped 300 jobs [ 85% ]  [  1222 / 1240  ]
Pausing for 0 seconds
-------------------------
 
Finished Scraping 350 jobs [ 100% ]  [  1222 / 1240  ]  [  Total: 114098 jobs  ]
 
 
-------------------------

There are 156 jobs for your [  1223 / 1240  ] search!

link of this search: https://www.indeed.com/jobs?q=people+analyst&l=oakland&radius=5&jt=all&filter=0&limit=50

Initializing Scraping 156 People analyst jobs in Oakland!  [  1223 / 1240  ]
 
-------------------------
Scraped 50 jobs [ 32% ]  [  1223 / 1240  ]
Pausing for 0 seconds
-------------------------
Scraped 100 jobs

-------------------------

There are 24 jobs for your [  1235 / 1240  ] search!

link of this search: https://www.indeed.com/jobs?q=people+analyst&l=walnut+creek&radius=5&jt=all&filter=0&limit=50

Initializing Scraping 24 People analyst jobs in Walnut Creek!  [  1235 / 1240  ]
 
-------------------------
 
Finished Scraping 24 jobs [ 100% ]  [  1235 / 1240  ]  [  Total: 115820 jobs  ]
 
 
-------------------------

There are 45 jobs for your [  1236 / 1240  ] search!

link of this search: https://www.indeed.com/jobs?q=people+analyst&l=fremont&radius=5&jt=all&filter=0&limit=50

Initializing Scraping 45 People analyst jobs in Fremont!  [  1236 / 1240  ]
 
-------------------------
 
Finished Scraping 45 jobs [ 100% ]  [  1236 / 1240  ]  [  Total: 115865 jobs  ]
 
 
-------------------------

There are 121 jobs for your [  1237 / 1240  ] search!

link of this search: https://www.indeed.com/jobs?q=people+analyst&l=milpitas&radius=5&jt=all&filter=0&limit=50

Initializing Scraping 121 People

# Phase 2: Scraping the body of Job Posts

In [4]:
df_final = df1

p2_start = time.time()

# Loop to request job links collected from the previous phase
target_body = df_final.shape[0]
df_body = pd.DataFrame(columns=["Body"])

for job_link in range(0, target_body, 1):
    job_url = df_final.Link.iloc[job_link]

    if job_link is not 0:
        job_progress =  int(job_link/target_body * 100)
        print "Scraped the Body of " + str(job_link) + " jobs [ " + str(job_progress) + "% ]  [ " + str(job_link) + " / " +  str(target_body) + " ]"

    else:
        print "Initializing Scraping the Body of " + str(target_body) + " jobs!"
        print " "
    html = requests.get(job_url)
    soup = BeautifulSoup(html.content, 'html.parser', from_encoding="utf-8")

    # Scrape the body of job link

    try:
        body = soup.select('.snip')
        idx = body[0].getText().find('save job')
        body = body[0].getText()[0:idx]
    except:
        
        body = "None"

    # Append the scraped body in a Data Frame           

    df_body = df_body.append({'Body': body}, ignore_index=True)

    # Pausing the loop for a random value in a normal distribution with a mean of 1 seconds, imitate human browsing     

    pausetime = int(np.abs(np.random.randn(1) + 0))  
    if job_link <> 0:
        print "Pausing for " + str(pausetime) + " seconds"

        print "-------------------------"
        time.sleep(pausetime)

progress =  int((job_link+1)/target_body * 100)

print " "
print "Finished Scraping " + str(df_final.shape[0]) + " jobs [ " + str(progress) + "% ]"
print " "
print "-------------------------"
print "-------------------------"
print " "
print " "
# Merging the old and new Data Frames

df_final = pd.merge(df_final, df_body, left_index=True, right_index=True)

scraped = df_final.shape[0]

print "Total jobs scraped: " + str(scraped) + " jobs."


df_final.drop_duplicates(keep='first', subset="Body", inplace=True)

duplicates  = scraped - df_final.shape[0]

print "We found " + str(duplicates) + " duplicated jobs that we removed." 
print "So we only saved " + str(df_final.shape[0]) + " jobs."

df_final.reset_index(drop=True, inplace=True)

p2_duration = time.strftime("%H:%M:%S", time.gmtime(time.time() - p2_start))

df2 = df_final
print "Duration: " + p2_duration


Initializing Scraping the Body of 9644 jobs!
 
Scraped the Body of 1 jobs [ 0% ]  [ 1 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 2 jobs [ 0% ]  [ 2 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 3 jobs [ 0% ]  [ 3 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 4 jobs [ 0% ]  [ 4 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 5 jobs [ 0% ]  [ 5 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 6 jobs [ 0% ]  [ 6 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 7 jobs [ 0% ]  [ 7 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 8 jobs [ 0% ]  [ 8 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9 jobs [ 0% ]  [ 9 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 10 jobs [ 0% ]  [ 10 / 9644 ]
Pausing for 0 seconds
-----------------

Pausing for 0 seconds
-------------------------
Scraped the Body of 85 jobs [ 0% ]  [ 85 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 86 jobs [ 0% ]  [ 86 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 87 jobs [ 0% ]  [ 87 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 88 jobs [ 0% ]  [ 88 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 89 jobs [ 0% ]  [ 89 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 90 jobs [ 0% ]  [ 90 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 91 jobs [ 0% ]  [ 91 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 92 jobs [ 0% ]  [ 92 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 93 jobs [ 0% ]  [ 93 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 94 jobs [ 0% ]  [ 94 / 9644 ]
Pausing for 0 second

Pausing for 2 seconds
-------------------------
Scraped the Body of 168 jobs [ 1% ]  [ 168 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 169 jobs [ 1% ]  [ 169 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 170 jobs [ 1% ]  [ 170 / 9644 ]
Pausing for 2 seconds
-------------------------
Scraped the Body of 171 jobs [ 1% ]  [ 171 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 172 jobs [ 1% ]  [ 172 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 173 jobs [ 1% ]  [ 173 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 174 jobs [ 1% ]  [ 174 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 175 jobs [ 1% ]  [ 175 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 176 jobs [ 1% ]  [ 176 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 177 jobs [ 1% ]  [ 177 / 9644 ]


Pausing for 0 seconds
-------------------------
Scraped the Body of 250 jobs [ 2% ]  [ 250 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 251 jobs [ 2% ]  [ 251 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 252 jobs [ 2% ]  [ 252 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 253 jobs [ 2% ]  [ 253 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 254 jobs [ 2% ]  [ 254 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 255 jobs [ 2% ]  [ 255 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 256 jobs [ 2% ]  [ 256 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 257 jobs [ 2% ]  [ 257 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 258 jobs [ 2% ]  [ 258 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 259 jobs [ 2% ]  [ 259 / 9644 ]


Pausing for 1 seconds
-------------------------
Scraped the Body of 332 jobs [ 3% ]  [ 332 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 333 jobs [ 3% ]  [ 333 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 334 jobs [ 3% ]  [ 334 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 335 jobs [ 3% ]  [ 335 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 336 jobs [ 3% ]  [ 336 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 337 jobs [ 3% ]  [ 337 / 9644 ]
Pausing for 2 seconds
-------------------------
Scraped the Body of 338 jobs [ 3% ]  [ 338 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 339 jobs [ 3% ]  [ 339 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 340 jobs [ 3% ]  [ 340 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 341 jobs [ 3% ]  [ 341 / 9644 ]


...

Pausing for 0 seconds
-------------------------
Scraped the Body of 9308 jobs [ 96% ]  [ 9308 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9309 jobs [ 96% ]  [ 9309 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9310 jobs [ 96% ]  [ 9310 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9311 jobs [ 96% ]  [ 9311 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9312 jobs [ 96% ]  [ 9312 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9313 jobs [ 96% ]  [ 9313 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9314 jobs [ 96% ]  [ 9314 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9315 jobs [ 96% ]  [ 9315 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9316 jobs [ 96% ]  [ 9316 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9317 

Pausing for 1 seconds
-------------------------
Scraped the Body of 9388 jobs [ 97% ]  [ 9388 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9389 jobs [ 97% ]  [ 9389 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9390 jobs [ 97% ]  [ 9390 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9391 jobs [ 97% ]  [ 9391 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 9392 jobs [ 97% ]  [ 9392 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9393 jobs [ 97% ]  [ 9393 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9394 jobs [ 97% ]  [ 9394 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 9395 jobs [ 97% ]  [ 9395 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9396 jobs [ 97% ]  [ 9396 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9397 

Pausing for 1 seconds
-------------------------
Scraped the Body of 9468 jobs [ 98% ]  [ 9468 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 9469 jobs [ 98% ]  [ 9469 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9470 jobs [ 98% ]  [ 9470 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9471 jobs [ 98% ]  [ 9471 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9472 jobs [ 98% ]  [ 9472 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9473 jobs [ 98% ]  [ 9473 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9474 jobs [ 98% ]  [ 9474 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 9475 jobs [ 98% ]  [ 9475 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 9476 jobs [ 98% ]  [ 9476 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9477 

Pausing for 0 seconds
-------------------------
Scraped the Body of 9548 jobs [ 99% ]  [ 9548 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 9549 jobs [ 99% ]  [ 9549 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9550 jobs [ 99% ]  [ 9550 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9551 jobs [ 99% ]  [ 9551 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9552 jobs [ 99% ]  [ 9552 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9553 jobs [ 99% ]  [ 9553 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9554 jobs [ 99% ]  [ 9554 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 9555 jobs [ 99% ]  [ 9555 / 9644 ]
Pausing for 1 seconds
-------------------------
Scraped the Body of 9556 jobs [ 99% ]  [ 9556 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9557 

Pausing for 0 seconds
-------------------------
Scraped the Body of 9628 jobs [ 99% ]  [ 9628 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9629 jobs [ 99% ]  [ 9629 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9630 jobs [ 99% ]  [ 9630 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9631 jobs [ 99% ]  [ 9631 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9632 jobs [ 99% ]  [ 9632 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9633 jobs [ 99% ]  [ 9633 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9634 jobs [ 99% ]  [ 9634 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9635 jobs [ 99% ]  [ 9635 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9636 jobs [ 99% ]  [ 9636 / 9644 ]
Pausing for 0 seconds
-------------------------
Scraped the Body of 9637 

# Phase 3: Text mining the body of Job Posts

In [5]:
df_final = df2

p3_start = time.time()

print "Initializing extracting keywords from Body of " + str(df_final.shape[0]) + " jobs!"
print " "
print "-------------------------"
columns = []
dictionary_count = len(keywords.keys())
#creating columns for keywords
for x in range(0, dictionary_count, 1):
    
    for y in range(1, len(keywords.get(list(keywords)[x])), 1):
        keyword = keywords.get(list(keywords)[x])[0] + seperator + keywords.get(list(keywords)[x])[y]
        columns.append(keyword)
        
df_exp = pd.DataFrame(columns= ['Experience', 'Exp Min', 'Exp Max'])

df_text = pd.DataFrame(columns= [columns] , index=df_final.index)

keyword_count = len(list(df_text.columns.values))

#Experience
for row in range(0, df_final.shape[0], 1):
    try:
        idx = df_final.Body[row].find('years')
        experience = df_final.Body[row][idx-10:idx+5]
        exp = re.sub('\D', '', experience)
        experience = df_final.Body[row][idx-35:idx+35]
        if len(exp) > 4 or exp == "":
            experience = None
            expmin = None
            expmax = None
        else:
            if len(exp) == 4:
                expmin = exp[0:2]
                expmax = exp[2:4]
            if len(exp) == 3:
                expmin = exp[0]
                expmax = exp[1:3]
            if len(exp) == 2:
                if exp[1] > exp[0]:
                    expmin = exp[0]
                    expmax = exp[1]
                else:
                    expmin = exp
                    expmax = None
                
            if len(exp) == 1:
                expmin = exp
                expmax = None
            
            if int(expmin) > 20:
                expmin = None
                expmax = None
                experience = None
            if expmax is not None and (int(expmax) > 20 or int(expmax) <= int(expmin)):
                expmin = None
                expmax = None
                experience = None
    except:
        experience = None
        expmin = None
        expmax = None
    
    df_exp = df_exp.append({'Experience': experience, 'Exp Min': expmin, 'Exp Max' : expmax}, ignore_index=True)
    
#Keywords    
    for key in range(0, keyword_count, 1):
            word = list(df_text.columns.values)[key].split("_")[1]
            idx = df_final.Body[row].find(word)
            
            if word in df_final.Body.iloc[row] or (len(word) <> 2 and word.lower() in df_final.Body.iloc[row]) :
                if df_final.Body.iloc[row][idx-1] is not "H":
                    df_text.iloc[row,key] = 1            
            else:
                df_text.iloc[row,key] = 0 
                
    progress = int(row / (df_final.shape[0] - 1) * 100)
    if (row+1) % 250 == 0:
        print "Extracted keywords from the Body of " + str(row + 1) + " jobs [ " + str(progress) + "% ]" 
        print "-------------------------"

print " "
print " "
print "Finished extracting keywords from Body of " + str(df_final.shape[0]) + " jobs [ " + str(progress) + "% ]"

df_final = pd.merge(df_final, df_exp, left_index=True, right_index=True)
df_final = pd.merge(df_final, df_text, left_index=True, right_index=True)

p3_duration = time.strftime("%H:%M:%S", time.gmtime(time.time() - p3_start))

df3 = df_final
print "Duration: " + p3_duration

Initializing extracting keywords from Body of 9204 jobs!
 
-------------------------
Extracted keywords from the Body of 250 jobs [ 2% ]
-------------------------
Extracted keywords from the Body of 500 jobs [ 5% ]
-------------------------
Extracted keywords from the Body of 750 jobs [ 8% ]
-------------------------
Extracted keywords from the Body of 1000 jobs [ 10% ]
-------------------------
Extracted keywords from the Body of 1250 jobs [ 13% ]
-------------------------
Extracted keywords from the Body of 1500 jobs [ 16% ]
-------------------------
Extracted keywords from the Body of 1750 jobs [ 19% ]
-------------------------
Extracted keywords from the Body of 2000 jobs [ 21% ]
-------------------------
Extracted keywords from the Body of 2250 jobs [ 24% ]
-------------------------
Extracted keywords from the Body of 2500 jobs [ 27% ]
-------------------------
Extracted keywords from the Body of 2750 jobs [ 29% ]
-------------------------
Extracted keywords from the Body of 3000 

# Phase 4: Classifying jobs

In [6]:
df_final = df3

p4_start = time.time()

df_title = pd.DataFrame(columns= ["General_Title", "Department", "City"])
df_final = df3
for job in  range(0, df_final.shape[0], 1):
    point = df_final.Title.iloc[job]
    GT = ''
    pre = ''
    data = ''
    post = 'Analyst'
    dep = 'General'
    if 'Senior ' in point or 'Sr' in point:
        pre = 'Senior '
#Data
    if 'Data' in point:
        data = 'Data '
        dep = 'Statistics and Data Science'
#Scientist
    if 'Scientist' in point or 'Science' in point :
        post = 'Scientist'
        dep = 'Statistics and Data Science'
        data = 'Data '
#Business
    if 'Business' in point or 'BI' in point or 'Intelligence' in point:
        GT = 'Business '
        dep = 'Business'
        
#IT
    if 'IT' in point or 'System' in point or 'Security' in point or 'Engineer' in point or \
    'Technical' in point or 'Architect' in point or 'SOX' in point or 'Technology' in point:        
        GT = 'IT '  
        dep = 'IT'
        
#Marketing
    if 'Marketing' in point or 'SEO' in point or 'SEM' in point or 'Campaign' in point or \
    'Product' in point or 'Digital' in point or 'Media' in point or 'Growth' in point or 'Engagement' in point:            
        GT = 'Marketing '
        dep = 'Marketing'
        
#Supply Chain    
    if 'Supply Chain' in point or 'Logistics' in point or 'Operations' in point or 'Procurement' in point:        
        GT = 'Supply Chain ' 
        dep = 'Supply Chain'
        
#Finance
    if 'Finance' in point or 'Financial' in point or 'Asset' in point or 'Accounting' in point or \
    'Equity' in point or 'Investment' in point or 'Portfolio' in point or 'Banking' in point or \
    'Credit' in point or 'Risk' in point or 'Venture' in point or 'VC' in point or \
    'Securities' in point or 'Fund' in point or 'Investor' in point or 'Venture' in point or \
    'Capital' in point or 'Revenue' in point or 'Loan' in point or 'Wealth' in point or 'FinTech' in point or \
    'Tax' in point:
        GT = 'Financial '
        dep = 'Finance'
#Sales
    if 'Sales' in point or 'Operations' in point or 'Account' in point or 'Channel' in point or \
    'Partner' in point or 'Customer' in point or 'Relationship' in point or 'CRM'in point:
        GT = 'Sales '
        dep = 'Sales'
#HR       
    if 'HR' in point or 'Human Resources' in point or 'People' in point or 'Staff' in point or \
    'Organizational' in point or 'OD' in point or 'Talent' in point or 'Compensation' in point or \
    'Rewards' in point or 'Payroll' in point or 'Recruiting' in point or 'Benefit' in point:
        GT = 'HR '
        dep = 'HR'
           
#City
    idx = df_final.Location.iloc[job].find(',')
    city = df_final.Location.iloc[job][:idx]    
    
#Progress        
    progress = int(job / (df_final.shape[0] - 1) * 100)

    if job % 250 == 0:
        print "Filtered " + str(job) + " jobs [ " + str(progress) + "% ]" 
        print "-------------------------"        

    df_title = df_title.append({'General_Title':(pre + GT + data + post), 'Department': dep , 'City' : city}, ignore_index=True)

df_final = pd.merge(df_title, df_final, left_index=True, right_index=True)

print " "
print " "
print "Finished filtering " + str(job) + " jobs [ " + str(progress) + "% ]"
print "-------------------------" 

p4_duration = time.strftime("%H:%M:%S", time.gmtime(time.time() - p4_start))

end_time = strftime("%Y-%m-%d %H:%M:%S", gmtime())

end = time.time()

total_time = time.strftime("%H:%M:%S", time.gmtime(end - start))

df4 = df_final
print 'We started at ' + start_time
print 'Phase 1 took ' + p1_duration
print 'Phase 2 took ' + p2_duration
print 'Phase 3 took ' + p3_duration
print 'Phase 4 took ' + p4_duration
print 'We finished at ' + end_time
print 'The code ran for ' + total_time

# export dataset
file_date = strftime("%m-%d-%Y", gmtime())
df_final.to_excel('dataset-' + file_date +'.xls')

Filtered 0 jobs [ 0% ]
-------------------------
Filtered 250 jobs [ 2% ]
-------------------------
Filtered 500 jobs [ 5% ]
-------------------------
Filtered 750 jobs [ 8% ]
-------------------------
Filtered 1000 jobs [ 10% ]
-------------------------
Filtered 1250 jobs [ 13% ]
-------------------------
Filtered 1500 jobs [ 16% ]
-------------------------
Filtered 1750 jobs [ 19% ]
-------------------------
Filtered 2000 jobs [ 21% ]
-------------------------
Filtered 2250 jobs [ 24% ]
-------------------------
Filtered 2500 jobs [ 27% ]
-------------------------
Filtered 2750 jobs [ 29% ]
-------------------------
Filtered 3000 jobs [ 32% ]
-------------------------
Filtered 3250 jobs [ 35% ]
-------------------------
Filtered 3500 jobs [ 38% ]
-------------------------
Filtered 3750 jobs [ 40% ]
-------------------------
Filtered 4000 jobs [ 43% ]
-------------------------
Filtered 4250 jobs [ 46% ]
-------------------------
Filtered 4500 jobs [ 48% ]
-------------------------
Fil

In [7]:
df_final

,General_Title,Department,City,Search,Title,Location,Company,Salary,Summary,Post_Date,...,PL_SPARK,PL_SPSS,PL_SQL,PL_Stata,PL_Swift,PL_Tcl,PL_VBScript,PL_Verilog,PL_VHDL,PL_Visual Basic
0,Analyst,General,San Francisco,Actuarial Analyst,Actuarial Analyst (Reserving),"San Francisco, CA 94103 (South Of Market area)",Uber,None,Casualty lines actuarial experienc...,23 days ago,...,0,0,1,0,1,0,0,0,0,0
1,Analyst,General,San Francisco,Actuarial Analyst,"Claims Analytics, Actuarial Analyst","San Francisco, CA 94103 (South Of Market area)",Uber,None,This analyst will partner closely ...,30+ days ago,...,0,0,1,0,1,0,0,0,0,0
2,Analyst,General,San Francisco,Actuarial Analyst,"Analyst, Actuarial I Associate","San Francisco, CA 94105 (Financial District area)",Blue Shield of California,None,Learns various actuarial technique...,28 days ago,...,0,0,0,0,0,0,0,0,0,0
3,Analyst,General,San Francisco,Actuarial Analyst,Reinsurance Analyst - San Francisco,"San Francisco, CA",Willis Towers Watson,None,Willis Re currently has an opening...,30+ days ago,...,0,0,0,0,0,0,0,0,0,0
4,Analyst,General,San Francisco,Actuarial Analyst,Quantitative Analyst,"San Francisco, CA",CyberCube Analytics,None,Bachelor’s degree in a related qua...,30+ days ago,...,0,0,1,0,0,0,0,0,0,0
5,Analyst,General,San Francisco,Actuarial Analyst,GIS Analyst - Intern,"San Francisco, CA 94108 (Financial District area)",Milliman,None,Seeks to hire a GIS Analyst Intern...,1 day ago,...,0,0,1,0,0,0,0,0,0,0
6,Financial Analyst,Finance,San Francisco,Actuarial Analyst,Healthcare Financial Analyst,"San Francisco, CA",Health Plan of San Mateo,None,"Bachelor’s degree in Economics, Fi...",30+ days ago,...,0,0,0,0,0,0,0,0,0,0
7,Analyst,General,San Francisco,Actuarial Analyst,"Analyst, Actuarial III Senior","San Francisco, CA 94105 (Financial District area)",Blue Shield of California,None,Typically requires four years of p...,9 days ago,...,0,0,0,0,0,0,0,0,0,0
8,Analyst,General,San Francisco,Actuarial Analyst,"Intermediate Analyst, Actuarial","San Francisco, CA 94105 (Financial District area)",Blue Shield of California,None,NTA is seeking an Intermediate Act...,30+ days ago,...,0,0,1,0,0,0,0,0,0,0
9,Senior HR Analyst,HR,San Francisco,Actuarial Analyst,Health and Benefits Senior Analyst,"San Francisco, CA",Willis Towers Watson,None,"Pursuit of CEBS designation, or he...",30+ days ago,...,0,0,0,0,0,0,0,0,0,0
